In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType


In [2]:
spark = (SparkSession.builder
		 .appName('df_ex')
		 .getOrCreate())

24/01/24 12:49:06 WARN Utils: Your hostname, geoffroy-XPS-15-9520 resolves to a loopback address: 127.0.1.1; using 192.168.13.63 instead (on interface wlp0s20f3)
24/01/24 12:49:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/24 12:49:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
file = 'data/sparkify_log_small.json'
df = spark.read.json(file)

In [7]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)


# Q1
which page did user id "" (empty string) NOT visit?

In [9]:
df.filter(df.userId == '') \
	.select('page') \
    .dropDuplicates() \
	.show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+


# Q2
How many female users do we have in the data set?

In [36]:
df.filter(df.gender == 'F') \
	.select(['userId', 'gender']) \
	.dropDuplicates() \
	.count()

462

# Q3
How many songs were played from the most played artist?

In [21]:
df.groupBy('artist').count().orderBy('count', ascending=False).show(5)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                NULL| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
+--------------------+-----+


# Q4
How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [35]:
df.filter(df.userId != '').groupBy(['userId', 'page']).count().agg(F.avg('count')).collect()

[Row(avg(count)=5.487791027825099)]